In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# i am importing the libraries that will be used

In [ ]:
import plotly.graph_objects as go
from plotly.offline import iplot
import folium
from  folium  import  plugins

# Loading the datas

In [ ]:
resultsDF = pd.read_csv('/kaggle/input/formula-1-world-championship-1950-2020/results.csv')
circuitsDF = pd.read_csv('/kaggle/input/formula-1-world-championship-1950-2020/circuits.csv')
driversDF = pd.read_csv('/kaggle/input/formula-1-world-championship-1950-2020/drivers.csv')
racesDF = pd.read_csv('/kaggle/input/formula-1-world-championship-1950-2020/races.csv')
constructorDF = pd.read_csv('/kaggle/input/formula-1-world-championship-1950-2020/constructors.csv')


# showing the contents of the Dataframes

In [ ]:
resultsDF.head()



In [ ]:
circuitsDF.head()

In [ ]:
driversDF.head()

In [ ]:
racesDF.head()

In [ ]:
constructorDF.head()

# joining the Dataframes

In [ ]:
#joining resulDF with driversDF by column driverID,using inner join
dfresul = pd.merge(resultsDF,driversDF,on='driverId')
#joining racesDF with dfresul by column raceID,using inner join
dfresulcons = pd.merge(dfresul,racesDF,on='raceId')
#joining constructorDF with dfresulcons by column constructorID,using inner join
dfresulrac = pd.merge(dfresulcons,constructorDF,on='constructorId')



# showing the dataframe after the joins

In [ ]:
dfresulrac.head()

# checking for null values

In [ ]:
dfresulrac.isnull().sum()

# deleting unnecessary and repeated columns

In [ ]:
dfresulrac = dfresulrac.drop(columns=['url_x','url_y','name_y','nationality_y','url','time_y'])
dfresulrac.head()

# showing all circuits that have had races through a heat map

In [ ]:
coordinates=[]
for lat,lng in zip(circuitsDF['lat'],circuitsDF['lng']):
  coordinates.append([lat,lng])

In [ ]:
maps = folium.Map(location=[-15.788497,-47.879873],zoom_start=2,tiles='Stamen Toner')

In [ ]:
maps.add_child(plugins.HeatMap(coordinates))
maps

# driver's championship ranking by season

In [ ]:
def championship_year(year):
    classi = dfresulrac[(dfresulrac['year']== year)].groupby('driverRef')['points'].sum().reset_index()
    classi = classi.sort_values(by=['points'],ascending=False)
    fig = go.Figure(data=[go.Bar(
    x=classi['driverRef'],
    y=classi['points']
)])
    fig.update_layout(title={
        'text': f"Driver´s championship ranking of {year}",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'}, 
                  yaxis=dict(
                            title='Points',
                            titlefont_size=16,
                            tickfont_size=14),
                  xaxis=dict(
                            title='Driver',
                            titlefont_size=16,
                            tickfont_size=14),
                  template = "plotly_dark"
                  )
    return iplot(fig)

In [ ]:
# choose the year to show the ranking: 1950 - 2021
championship_year(2007)

# Constructors' Championship ranking by season

In [ ]:
def championship_cons(year):
    classi_equp = dfresulrac[(dfresulrac['year']== year)].groupby('constructorRef')['points'].sum().reset_index()
    classi_equp = classi_equp.sort_values(by=['points'],ascending=False)
    fig = go.Figure(data=[go.Bar(
    x=classi_equp['constructorRef'],
    y=classi_equp['points']
    )])
    fig.update_layout(title={
        'text': f"Constructors' Championship ranking of {year}",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'}, 
                  yaxis=dict(
                            title='Points',
                            titlefont_size=16,
                            tickfont_size=14),
                  xaxis=dict(
                            title='Constructor',
                            titlefont_size=16,
                            tickfont_size=14),
                  template = "plotly_dark"
                  )
    return iplot(fig)

In [ ]:
# choose the year to show the ranking: 1950 - 2021
championship_cons(2013)

# drivers with more wins

In [ ]:
num_wins = dfresulrac[(dfresulrac['position']== '1')]
num_wins['position_mod'] = num_wins['position'].astype(int)
num_wins = num_wins.groupby(['driverRef','nationality_x'])['position_mod'].sum().reset_index()
num_wins = num_wins.sort_values(by=['position_mod'], ascending=False)

In [ ]:
fig = go.Figure(data=[go.Bar(
    x= num_wins['driverRef'],
    y= num_wins['position_mod'],
    hovertext = num_wins['nationality_x']
)])
fig.update_layout(title={
        'text': "Drivers with more wins",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'}, 
                  yaxis=dict(
                            title='wins',
                            titlefont_size=16,
                            tickfont_size=14),
                  xaxis=dict(
                            title='Driver',
                            titlefont_size=16,
                            tickfont_size=14),
                  template = "plotly_dark"
                  )
iplot(fig)

# drivers with more titles.
ps: the result points out as champion the driver who scored the most in a season, that's why prost appears with 5 titles when he actually has 4 and senna with 2 when he actually has 3, due to the regulation of the 1988 championship, which drivers discard the worst results , hamilton has 8 titles because he is the leader of the championship until the data is updated

In [ ]:
sum_driver = dfresulrac.groupby(['year','driverRef'])['points'].sum().reset_index()
champions = sum_driver.loc[sum_driver.reset_index().groupby(['year'])['points'].idxmax()]
champions = champions['driverRef'].value_counts().reset_index()
champions.rename(columns={'index':'driver','driverRef':'titles'}, inplace = True)

In [ ]:
fig = go.Figure(data=[go.Bar(
    x=champions['driver'],
    y=champions['titles']
    )])
fig.update_layout(title={
        'text': "Drivers with more titles",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'}, 
                  yaxis=dict(
                            title='Titles',
                            titlefont_size=16,
                            tickfont_size=14),
                  xaxis=dict(
                            title='Driver',
                            titlefont_size=16,
                            tickfont_size=14),
                  template = "plotly_dark"
                  )
iplot(fig)

# drivers with more pole positions

In [ ]:
driver_pole = dfresulrac[dfresulrac['grid']== 1].groupby(by=['driverRef','nationality_x'])['grid'].sum().reset_index()
driver_pole = driver_pole.sort_values(by=['grid'], ascending=False)

In [ ]:
fig = go.Figure(data=[go.Bar(
    x=driver_pole['driverRef'],
    y=driver_pole['grid'],
    hovertext = driver_pole['nationality_x']
)])
fig.update_layout(title={
        'text': "Drivers with more pole positions",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'}, 
                  yaxis=dict(
                            title='Pole positions',
                            titlefont_size=16,
                            tickfont_size=14),
                  xaxis=dict(
                            title='Driver',
                            titlefont_size=16,
                            tickfont_size=14),
                  template = "plotly_dark"
                  )
iplot(fig)

# drivers with more podiums

In [ ]:
p1 = dfresulrac[(dfresulrac['position']== '1')].groupby('driverRef')['position'].value_counts()
p2 = dfresulrac[(dfresulrac['position']== '2')].groupby('driverRef')['position'].value_counts()
p3 = dfresulrac[(dfresulrac['position']== '3')].groupby('driverRef')['position'].value_counts()
#pilotos = dfresulrac.sort_values(by=['driverRef'])['driverRef'].unique()
driver1 = dfresulrac[(dfresulrac['position']== '1')].sort_values(by=['driverRef'])['driverRef'].unique()
driver2 = dfresulrac[(dfresulrac['position']== '2')].sort_values(by=['driverRef'])['driverRef'].unique()
driver3 = dfresulrac[(dfresulrac['position']== '3')].sort_values(by=['driverRef'])['driverRef'].unique()

In [ ]:
fig = go.Figure(go.Bar(x=driver1, y=p1, name='1º place'))
fig.add_trace(go.Bar(x=driver2, y=p2, name='2º place'))
fig.add_trace(go.Bar(x=driver3, y=p3, name='3º place'))

fig.update_layout(barmode='stack',title={
        'text': "driver with more podiums",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'}, 
                  yaxis=dict(
                            title='Podiums',
                            titlefont_size=16,
                            tickfont_size=14),
                  xaxis=dict(
                            title='Driver',
                            titlefont_size=16,
                            tickfont_size=14),
                  template = "plotly_dark"
                  )
iplot(fig)

# country with more wins

In [ ]:
fig = go.Figure(data=[go.Pie(labels=dfresulrac[(dfresulrac['position']== '1')].sort_values(by=['nationality_x'])['nationality_x'].unique(), values=dfresulrac[(dfresulrac['position']== '1')].groupby('nationality_x')['position'].value_counts(),hole=.3)])
fig.update_layout(title={
        'text': "Country with more wins",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'}, 
        template = "plotly_dark")
iplot(fig)

# I will update with analysis about fastest lap and pit stops